### TP 4 EX 1
Grupo 5:
     
     Breno Fernando Guerra Marrão A97768
     
     Tales André Rovaris Machado A96314

## Inicialização
Usamos a biblioteca pysmt para resolver o problema proposto 

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import *


## Inicialização de variáveis

In [2]:
START = Int(0)
FREE = Int(1) # No modo Free  não existe qualquer força de travagem
STOPPING = Int(2)
BLOCKED = Int(3) # no modo Blocked as rodas estão bloqueadas em relação ao corpo mas o veículo  move-se (i.e. derrapa)
STOPPED = Int(4) # no modo Stopped o veículo está imobilizado.

a = 5 # Forca de atrito
P = 100 #peso
c = 1
Vi = 60.0 # Velocidade inicial
T = 1 # Segundos maximo q pode ficar tanto no free como no blockd



Declararação das variáveis do FOTS correspondente ao veículo em que t é o tempo , m o estado que o veículo se encontra , V a velocidade do corpo em relação ao solo, V velocidade linear das rodas em relação ao solo.

In [3]:
def declare(i):
    s = {}
    s['t'] = Symbol('t'+str(i),REAL)
    s['m'] = Symbol('m'+str(i),INT)
    s['V'] = Symbol('V'+str(i),REAL)
    s['v'] = Symbol('v'+str(i),REAL)
    s['c'] = Symbol('cronometro'+str(i),REAL)

    return s

O estado inicial do FOTS é o seguinte :
$$
m = \mathsf{START}\wedge v = V_{0}  \wedge V_{0} = V \wedge t = 0 \wedge V_{0}  >= 0 \wedge c = 0 \\
$$
Temos que começar no Start e definimos V e v com o valor de V0 que é o "input" do problema , e que diz a velocidade inicial do veículo, e c que vai marcar a diferença de tempo .

In [2]:
def init(s):
    return And(Equals(s['m'],START),Equals(s['V'],Real(60)),s['V'] >= 0.0,Equals(s['V'],s['v']),Equals(s['t'], Real(0)),Equals(s['c'], Real(0)))


As transiçõe "untimed" do fots são as seguintes : 
$$
\begin{array}{c}
m = \mathsf{START} \wedge m' = \mathsf{FREE} \wedge v' = v \wedge V' = V \wedge t' = t \wedge c' = c \\
\vee\\
m = \mathsf{FREE} \wedge m' = \mathsf{STOPPING} \wedge v' = v \wedge V' = V \wedge t' = t \wedge c = T \wedge c' = 0  \\
\vee\\
m = \mathsf{FREE} \wedge m' = \mathsf{STOPPING} \wedge v' = v \wedge V' = V \wedge t' = t \wedge V = 0 \wedge c' = 0  \\
\vee\\
m = \mathsf{STOPPING} \wedge m' = \mathsf{STOPPED} \wedge v' = v \wedge V' = V \wedge t' = t \wedge v = 0 \wedge V = 0 \wedge c' = 0 \\
\vee\\
m = \mathsf{STOPPING} \wedge m' = \mathsf{BLOCKED} \wedge v' = v \wedge V' = V \wedge t' = t \wedge v = 0 \wedge V >= 0 \wedge c' = 0 \\
\vee\\
m = \mathsf{BLOCKED} \wedge m' = \mathsf{FREE} \wedge v' = v \wedge V' = V \wedge t' = t \wedge c' = 0 \wedge c = T \\
\vee\\
\end{array}
\\
$$
As transições "timed " -  switches do fots são as seguintes :
$$
\begin{array}{c}
m = \mathsf{FREE} \wedge m' = \mathsf{FREE} \wedge v' < v \wedge t' < t \wedge c < T \wedge V' < V \wedge v' \geq 0 \wedge V' \geq 0 \wedge  \\
\vee\\
\end{array}

In [28]:
def trans(s,p):
    #Untimed
    #Falta colocar as alterações de velocidade
    #Definir condições para trocar entre as variaveis que vai no timed ................
    #Stopping momento até bloquear definir
    startFree = And(Equals(s['m'],START),Equals(p['m'],FREE), Equals(s['v'],p['v']) ,Equals(s['V'],p['V']),Equals(s['c'],p['c'])) 
    freeStopping = And(Equals(s['m'],FREE),Equals(p['m'],STOPPING),Equals(s['v'],p['v']),Equals(p['c'],real(0)),Equals(s['V'],p['V'])
                       ,Equals(p['t'],s['t']),Equals(s['c'], Real(T)))
    freeStopping1 = And(Equals(s['m'],FREE),Equals(p['m'],STOPPING),Equals(s['v'],p['v']),Equals(p['c'],real(0)),
                        Equals(s['V'],real(0)),Equals(s['V'],p['V']),Equals(p['t'],s['t']))
    #rever esta condição stoppingBlocked = And(Equals(s['m'],STOPPING),Equals(p['m'],BLOCKED), Equals(s['v'],p['v']),
                          Equals(p['v'],Real(0),) ,Equals(s['V'],p['V']))
    stoppingStopped = And(Equals(s['m'],STOPPING),Equals(p['m'],STOPPED), Equals(s['v'],p['v']) ,Equals(s['V'],p['V'])
                          ,Equals(s['V'],Real(0)), Equals(s['v'],Real(0)),GT(s['V'],Real(0)),Equals(p['c'],Real(0)))
    blockedFree =  And(Equals(s['m'],BLOCKED),Equals(p['m'],FREE), Equals(s['v'],p['v']) ,Equals(p['c'],Real(0)),
                       Equals(s['V'],p['V']),Equals(s['c'],Real(0)))
    blockedStopping =  And(Equals(s['m'],BLOCKED),Equals(p['m'],Stopping), Equals(s['v'],p['v']) Equals(p['c'],Real(0)),
                       Equals(s['V'],p['V']),(Equals(s['V'],Real(0))), (Equals(s['v'],Real(0))))

    
    #DAR PRO FreeFree uma condição pra ele parar de fazer freeFree sem ser a do timer 
    #provavel colocar alguma porcentagem de diferenca de vel q nem no termostato 
    #Timed - Switchs
    freeFree = And(Equals(s['m'],FREE), Equals(p['m'],FREE),)
    blockedBlocked = And(Equals(s['m'],BLOCKED),cond(s['V'],s['v'],s['t'],p['V'],p['v'],p['t']),
                         Equals(p['m'],BLOCKED),Not(Equals(p['v'],Real(0))),GT(p['t'] , s['t']),
                         Equals(p['V'] ,p['v'])) 
    stoppingStopping = And(Equals(s['m'],STOPPING),Equals(p['m'],STOPPING),Not(Equals(p['v'],Real(0))),
                    GT(p['t'] , s['t']),cond(s['V'],s['v'],s['t'],p['V'],p['v'],p['t']))
    
    return Or(startFree ,freeStopping,stoppingBlocked ,blockedFree ,stoppingStopped,freeFree,blockedBlocked,stoppingStopping)
def cond (V0,v0,t0,V,v,t):
    return And(Equals(V-V0,-(V-v) * (t - t0)),Equals(v-v0,(-a*P + (c * (V-v))) * (t - t0)))



In [29]:
def gera_traco(declare,init,trans,k):
    with Solver(name="z3") as s:
        
        # completar
        trace = [declare(i) for i in range(k)]
        
        #semantics
        s.add_assertion(init(trace[0]))
        for i in range(k-1):
            s.add_assertion(trans(trace[i], trace[i+1]))
            
        if s.solve():
            '''
            m = s.get_model()
            for n, v in m:
                print(f'{n} = {v}')
            '''
            for i in range(k):
                print("Passo ",i)
                for v in trace[i]:
                    print(v, "=", s.get_value(trace[i][v]))
                print("------------------------")
    
                        
####
gera_traco(declare,init,trans,5)

SolverReturnedUnknownResultError: 

## 1. ”o veículo imobiliza-se completamente em menos de t segundos” 
Que pode ser traduzido por 
sendo T o número de segundos que queremos limitar sendo S o conjunto dos estados .
para qualquer estado se s_{t} >= T então V = 0 ^v= 0


## 2.“a velocidade V diminui sempre com o tempo”
Que pode ser traduzido por
Sendo S o conjunto dos estados .
Para qualquer i < length(s) : se s_{i}{t} < s_{i+1}{t} então s_{i}{V} > s_{i+1}{V}